<a href="https://colab.research.google.com/github/bnsreenu/python_for_microscopists/blob/master/331_fine_tune_SAM_mito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1 Import relevant libraries

In [ ]:
# import relevant libraries
import torch # for PyTorch framework
import torch.nn as nn # for defining layers of unet model and resizing
import torchvision.transforms.functional as TF # for ensuring dimensional alignment

### 2 Modelling U-Net model

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        '''
        Define how each building block of the U-Net model looks like
        '''
        super(DoubleConv, self).__init__()
        # define each block of the model
        # 2 convolutions with padding same, and after each convolution batch normalization and ReLU activation function
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1,  bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        # use the defined blocks to forward pass some input
        return self.conv(x)

class UNET(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, features=[64, 128, 256, 512]):
        '''
        Define all building blocks of the U-Net model (downblocks, bottom, upblocks, final_conv)
        '''
        # expects grayscale images as input, features are number of feature maps created in each block
        super(UNET, self).__init__()
        self.upblock = nn.ModuleList()
        self.downblock = nn.ModuleList()
        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)

        # define down part of U-Net
        # loop through the features and create the blocks
        # halve shape of feature maps and increase number of feature maps
        for feature in features:
            self.downblock.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # define up part of U-Net
        # loop through the features and create the blocks
        # double shape of feature maps and decrease number of feature maps
        for feature in reversed(features):
            self.upblock.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
            self.upblock.append(DoubleConv(feature*2, feature))

        # define bottom of the model
        self.bottom = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
    
    def forward(self, x):
        '''
        Define skip connections of the model and add them to the decoder part of the model
        '''
        skip_connections = []

        # safe skip connections of the encoder part of the model
        for down in self.downblock:
            x = down(x)
            skip_connections.append(x)
            x = self.pooling(x)

        # bottom of the model
        x = self.bottom(x)
        skip_connections = skip_connections[::-1]

        # add skip connections to the decoder part of the model
        for idx in range(0, len(self.ups), 2):
            x = self.upblock[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)